# Part 02 - Correct use of variable scope

This set of notes is about building a program from scratch (the development process) and learning about scope, one of the most confusing parts of programming for beginners. The "scope" is everything the computer can "see" at a particular point in the code, i.e. the names of variables and functions the computer has learnt (so far). We have to learn about scope, as many errors arise from not understanding it, and you want to keep your scope as minimal/clean as possible so that errors are easy to find. Also, scope is fundamental to understanding how code is read by the computer.

To motivate it, we are going to build a prime number calculator!

Prime numbers are natural numbers (positive integers) which have only themselves and 1 as factors. Thus 2 is a prime number and so is 3, but not 4 as $2\times2=4$. There are no simple patterns to the primes. In fact, as soon as a pattern appears it seems to be destroyed. For example, between the first two primes (2,3) it appears to be increasing by 1, but this is immediately broken by the next prime being 5. We might then hypothesise the increase in the primes to be +1,+2,+3 but then the next prime (7) is actually a +2 again! In fact, "seives" are one way of identifying primes by removing the patterns prime numbers create once they are discovered. For example, as the number 2 is prime, all other even numbers are no-longer prime. See the [seive of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes) for more on this.

There are some general statements we can make about primes, such as the probability of a number, $n$, being prime is proportional to $1/\ln(n)$ for sufficiently large $n$, but there is no fast way to test if a number is prime. Most approaches instead check if a number is *not* prime by looking for factors. If there are no integer factors then it is prime. This is known as [trial division](https://en.wikipedia.org/wiki/Trial_division), where given a candidate prime number $n$, we test every integer less than $n$ to see if it divides $n$ an integer number of times.

Lets try $n=5$ by hand,

In [2]:
print(5/1)
print(5/2)
print(5/3)
print(5/4)
print(5/5)

5.0
2.5
1.6666666666666667
1.25
1.0


As it is only divisible (with no remainder) by itself and one, 5 is indeed prime. Lets make this test smarter and get the computer to do more of the work for us, we want to test all numbers from $[1,\,n]$ automatically, so lets generate a list of numbers,

In [3]:
list(range(1,6))

[1, 2, 3, 4, 5]

We need to go to `6` as range misses out the last number. It does this for a good reason as range by default starts counting from zero, so `range(6)` gives you 6 numbers from $[0,5]$. We need to use the `list()` command here around the `range` call so when we print it we see all the values, otherwise it just stays as a range generator (see previous slides for more information).

Using a loop, we can now test all the divisors:

In [4]:
for divisor in range(1,5):
    print(5/divisor)

5.0
2.5
1.6666666666666667
1.25


The `for` loop goes through the list one by one and creates a variable `divisor` which points to each member of the list in turn for each iteration of the loop.

Now we can talk about about scope. After the loop, can the computer still see `divisor`? Can we still use it? Lets try  printing its value out:

In [5]:
print(divisor)

4


Yes! `divisor` did not disappear once the loop was complete. It sticks around with its last value (the last loop value). This means that `for` loops do not have their own scope but instead place any variables created inside them into the surrounding scope. 
As we're just directly running code, any variables created get placed in the *global* scope, which is visible everywhere in the code. We can check out what is in the scope at a particular point using the `dir()` command. So lets see what's in the global scope:

In [6]:
dir()

['In',
 'Out',
 '_',
 '_3',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 '_sh',
 'divisor',
 'exit',
 'get_ipython',
 'quit']

Some of the inner workings of python/jupyter are visible in the scope and can be easily identified as they start with underscores "_", so you can safely ignore those entries. As `dir` returns a list, we can check if something is in it,

In [7]:
"divisor" in dir()

True

OK, back to the problem at hand. When does this behaviour of `for` loops cause problems? The issue is that programmers tend to use the same names for variables over and over again (i.e., `x`, `n`, or `i`). Lets say you already had some code that uses a variable $n$ but only at its beginning and end. But later, you come back to the code and modify it to have a loop in the middle

In [8]:
n = 12
# Some long code here that makes you forget about n being used already
# ...

# New loop added later-on in the development process
x = 0
for n in range(8):
    x += n

# More long code that stops you seeing n being used again
# ...
print(n) # Originally this would print 12, but adding the loop makes it print 7!

7


Changing variables without realising it is the number one issue of new programmers in every language. Of course, one solution would be to always use a new long random name for each variable so they never get confused, but this makes your code unreadable. Sometimes it makes sense to use lots of variables called `x`!

So how do you get a new scope? One way is to make a function! Lets make a function that tests if a number is prime:

In [9]:
def isPrime(n):
    #Check all numbers between [2,n-1]
    for div in range(2,n):
        # The modulus operator % gives us the integer remainder of a division.
        if n % div == 0:
            # If there's no remainder, then the divisor divides n a whole number of times
            # So n must not be prime then, so immediately end the function and return false!
            return False
    # If we get here, then we've finished the loop and the number must be prime!
    return True

print("5 is prime:", isPrime(5))     # Should say True, 5 is prime!
print("15 is prime:", isPrime(15))   # Should say False, 15 is not prime!
print("div" in dir())                # Should be False, div is in the scope of the function, not the global scope!  

5 is prime: True
15 is prime: False
False


The last line checks if the loop variable `div` from inside `isPrime` is still in the global scope, and we see that it is not. If we try to use this variable in the global scope we will get an error.

What happens here exactly? When python enters a function, it makes a copy of the current global scope. The function can access everything in the global scope but any additional variables made while in this new scope are only visible while the function is running. Also, any new variables in the function scope "shadow" or override variables in the global scope with the same name (for the duration of the function). Once the function is finished, the new scope is thrown away and the old scope restored. As the variable `div` falls out of all scopes entirely, it is also deleted forever (called garbage collection). 

You should try to write code that is as neat and small as possible, where every variable appears exactly when its needed, and falls out of scope after you are done with it. This stops "spaghetti code" where everything is tied together and its difficult to see when/where a variable is used or changed.

This is why object orientation, functional programming, and every other paradigm of programming has been invented. Its all about making easily readable code, with very simple scopes so that no-one gets confused. One way to do this is to write functions that have everything they need as an argument, and return all output as a return value (no use of the global scope). Any function which alters variables in its surrounding scope is said to have "side effects", and its generally frowned upon (although many times it is unavoidable). In fact, languages like lisp/clojure actually don't allow side effects (kinda).

Lets summarise with some examples of scope,

In [12]:
x="global scope"  # Create a new variable x in the global sccope

def g():
    # All functions, when called, inherit a "copy" of the global scope, so this function
    # has access to x
    print("From inside g(), x is in the", x)

def f():
    # Here we make a new variable x
    x="f() scope"
    # This variable "shadows"/overrides the global x while we're inside the function scope
    print("From inside f(), x is in the", x)
    # Now the function ends, so we fall out of its scope and the local x is lost
   
f()
g()
print("From outside f(), x is in the", x) 

From inside f(), x is in the f() scope
From inside g(), x is in the global scope
From outside f(), x is in the global scope


Lets finish with a fully working example of the prime number program:

In [11]:
def isPrime(n):
    for div in range(2,n):
        if n % div == 0:
            return False
    return True

#Print the first primes in the range [0,100]
for n in range(2, 100):
    if isPrime(n):
        print(n)

2
3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97


## Extra credit

- Don't test even numbers for being prime, this should cut the python runtime in half (hint, use a slice on the `range(2,n)` in `isPrime` to eliminate even numbers by using an increment of 2).
- By the same logic, the smallest multiple of a prime number is 2, so only test for divisors up to $n/2$ (rounded down). Hint, python has a rounding down division, i.e. `n//2`.
- Actually we only need to test up to $\sqrt{n}$ as the largest factor. The reason is that any number $n$ factored into $n=a\times b$ can also be written as $n=b\times a$. The largest value that $\min(a,b)$ can be is $\sqrt{n}$. Modify your code to run up to this point. 
- Do you need to test all integers? If every number is prime, or a multiple of a prime number, then you only need to test if prime factors are integer divisors! Store a list of all currently discovered primes, and loop over those as potential divisors (up to $\sqrt{n}$ of course).
- Given a number $n$, the number of prime numbers with a value lower than $n$ tends to $n/\ln(n)$ as $n$ becomes large. Plot the exact answer and the approximation to figure out how large $n$ has to be for this to hold reasonably true.
